In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import time
import os
import pandas as pd
import random
from collections import deque
import numpy as np

In [2]:
def convert_edges_to_csv(edge_file):
    cleaned_data = []
    with open(edge_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("#") or line.startswith("%") or line == "":
                continue
            parts = line.split()
            try:
                if len(parts) >= 3:
                    u, v, w = int(parts[0]), int(parts[1]), float(parts[2])
                elif len(parts) == 2:
                    u, v = int(parts[0]), int(parts[1])
                    w = 1.0
                else:
                    continue
                cleaned_data.append([u, v, w])
            except ValueError:
                continue

    df = pd.DataFrame(cleaned_data, columns=['source', 'target', 'weight'])
    csv_file = edge_file.replace('.edges', '.csv')
    df.to_csv(csv_file, index=False)
    print(f"✅ Saved cleaned CSV: {csv_file}")


In [3]:
def convert_amazon_format_to_csv(edge_file):
    cleaned_data = []
    user_map = {}
    item_map = {}
    user_counter = 0
    item_counter = 1_000_000  # Offset to keep users/items distinct

    with open(edge_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith("#") or line.startswith("%") or line == "":
                continue
            parts = line.split(",")
            if len(parts) < 3:
                continue
            user_id, item_id, rating = parts[0], parts[1], parts[2]

            try:
                rating = float(rating)
            except ValueError:
                continue

            # Map user to int
            if user_id not in user_map:
                user_map[user_id] = user_counter
                user_counter += 1
            u = user_map[user_id]

            # Map item to int
            if item_id not in item_map:
                item_map[item_id] = item_counter
                item_counter += 1
            v = item_map[item_id]

            cleaned_data.append([u, v, rating])

    df = pd.DataFrame(cleaned_data, columns=['source', 'target', 'weight'])
    csv_file = edge_file.replace('.edges', '.csv')
    df.to_csv(csv_file, index=False)
    print(f"✅ Saved cleaned CSV: {csv_file}")


In [4]:
edge_files = [
    "rec-amz-books.edges",
    "rec-amz-Video-Games.edges",
    "rec-movielens.edges",
    "rec-movielens-tag-movies-10m.edges",
    "rec-movielens-user-movies-10m.edges"
]



In [5]:

for edge_file in edge_files:
    if "Video-Games" in edge_file or "books" in edge_file:
        convert_amazon_format_to_csv(edge_file)
    else:
        convert_edges_to_csv(edge_file)


KeyboardInterrupt: 

In [6]:
def load_edge_list(file_path):
    df = pd.read_csv(file_path)
    G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr='weight')
    return G


In [7]:
def bfs_sample(G, target_nodes):
    visited = set()
    queue = deque()
    start_node = random.choice(list(G.nodes()))
    queue.append(start_node)
    visited.add(start_node)

    while len(visited) < target_nodes and queue:
        current = queue.popleft()
        for neighbor in G.neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append(neighbor)
            if len(visited) >= target_nodes:
                break

    sampled_nodes = list(visited)
    return G.subgraph(sampled_nodes).copy()


In [8]:
def sample_connected_subgraph(G, target_nodes):
    if G.number_of_nodes() == 0:
        print("⚠️ Graph is empty. Skipping sampling.")
        return G

    if nx.is_connected(G):
        if G.number_of_nodes() <= target_nodes:
            return G
        else:
            return bfs_sample(G, target_nodes)

    # Graph is disconnected
    largest_cc = max(nx.connected_components(G), key=len)
    G_lcc = G.subgraph(largest_cc).copy()

    if G_lcc.number_of_nodes() < target_nodes:
        print(f"⚠️ Largest component only has {G_lcc.number_of_nodes()} nodes — using all of it.")
        return G_lcc

    return bfs_sample(G_lcc, target_nodes)


In [9]:
def reverse_delete_mst_with_steps(G):
    # Start with all edges in the MST candidate
    mst_edges = list(G.edges(data=True))
    mst_edges = [(u, v, d['weight']) for u, v, d in mst_edges]
    mst_edges.sort(key=lambda x: x[2], reverse=True)  # Sort edges descending by weight

    # Initially, MST graph is the full graph
    mst_graph = nx.Graph()
    mst_graph.add_nodes_from(G.nodes())
    mst_graph.add_weighted_edges_from(mst_edges)

    steps = [mst_edges.copy()]  # record all edges at start

    for edge in mst_edges[:]:  # iterate over a copy since we remove edges
        u, v, w = edge
        mst_graph.remove_edge(u, v)
        if not nx.is_connected(mst_graph):
            # Edge is necessary, put it back
            mst_graph.add_edge(u, v, weight=w)
        else:
            # Edge removed permanently, also remove from edge list
            mst_edges.remove(edge)
        steps.append(mst_edges.copy())  # record step after each removal attempt

    return mst_edges, steps


In [12]:
results = []

sample_size = {
    "rec-amz-books.csv": 800,
    "rec-amz-Video-Games.csv": 200,
    "rec-movielens.csv":1000,
    "rec-movielens-tag-movies-10m.csv": 400,
    "rec-movielens-user-movies-10m.csv": 600,
}

dataset_files = [f.replace(".edges", ".csv") for f in edge_files]

for dataset in dataset_files:
    print(f"\n📂 Processing: {dataset}")
    G = load_edge_list(dataset)

    sample_target = sample_size.get(os.path.basename(dataset))

    if sample_target is not None:
        print(f"📉 Sampling {sample_target} connected nodes...")
        G = sample_connected_subgraph(G, sample_target)
        print(f"✅ Sampled nodes: {G.number_of_nodes()}, edges: {G.number_of_edges()}")

    start_time = time.time()
    mst_edges, steps = reverse_delete_mst_with_steps(G)
    end_time = time.time()

    total_cost = sum(w for _, _, w in mst_edges)
    exec_time = end_time - start_time


    print(f"✅ MST total cost: {total_cost:.2f}")
    print(f"⏱️ Execution time: {exec_time:.2f} seconds")

    results.append({
        "Dataset": os.path.basename(dataset),
        "Nodes": G.number_of_nodes(),
        "Edges": G.number_of_edges(),
        "MST Cost": total_cost,
        "Execution Time (s)": exec_time
    })



📂 Processing: rec-amz-books.csv
📉 Sampling 800 connected nodes...
✅ Sampled nodes: 800, edges: 799
✅ MST total cost: 3699.00
⏱️ Execution time: 0.40 seconds

📂 Processing: rec-amz-Video-Games.csv
📉 Sampling 200 connected nodes...
✅ Sampled nodes: 200, edges: 208
✅ MST total cost: 755.00
⏱️ Execution time: 0.03 seconds

📂 Processing: rec-movielens.csv
📉 Sampling 1000 connected nodes...
✅ Sampled nodes: 1000, edges: 71771


MemoryError: 

In [ ]:
df_results = pd.DataFrame(results)
df_results_sorted = df_results.sort_values(by='Nodes', ascending=True)
df_results_sorted.reset_index(drop=True, inplace=True)
df_results_sorted


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def animate_mst_steps(G, steps, save_path="boruvka_animation.gif", max_frames=50):
    pos = nx.spring_layout(G, seed=42)  # Fixed layout
    fig, ax = plt.subplots(figsize=(6, 4))  # Smaller size

    def update(i):
        ax.clear()
        ax.set_title(f"Step {i+1}")
        ax.axis('off')
        step_edges = steps[min(i, len(steps)-1)]
        nx.draw_networkx_nodes(G, pos, node_size=10, ax=ax)
        nx.draw_networkx_edges(G, pos, edgelist=step_edges, edge_color='red', width=1, ax=ax)

    frames = min(len(steps), max_frames)
    ani = animation.FuncAnimation(fig, update, frames=frames, interval=500, repeat=False)

    ani.save(save_path, writer='pillow', fps=2)
    print(f"🎥 GIF saved to: {save_path}")


In [ ]:
# ⚙️ RUNNING MST + ANIMATION ON A SAMPLE DATASET
dataset = "rec-amz-books.csv"
G = load_edge_list(dataset)
G = sample_connected_subgraph(G, 300)  # Use smaller graph for animation

mst_edges, steps = reverse_delete_mst_with_steps(G)  # <-- change here
total_cost = sum(w for _, _, w in mst_edges)
print(f"📊 Reverse-Delete MST Cost: {total_cost:.2f}")  # <-- change here

animate_mst_steps(G, steps, save_path="reverse_delete_animation.gif")  # <-- change filename
